In [2]:
#ignore warnings for clear printing
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import seaborn as sns # More snazzy plotting library
import itertools


##test results 
#  1.looks the absolute year number gives a good colleration for prediction. 
#  2. standarlistaion improves prediction by 3%
#  3. removing aspect ratio improves prediction by 2%


#open the file
file_path = "/home/user/projects/data_mining/predictive_data_mining/logistic_regression/working_dataset.csv"
dta = pd.read_csv(file_path)

#clean up data non numeric rows
#str_list = [] # empty list to contain columns with strings (words)
#for colname, colvalue in dta.iteritems():
#    if type(colvalue[1]) == str:
        #if colname not in str_list:
#            str_list.append(colname)
# Get to the numeric columns by inversion
#num_list = dta.columns.difference(str_list)
#USe only the numeriv values
#dta_clean = dta[num_list]

#save all rows
dta_clean = dta
#remove the null values, that is fill NaN with there - FIXME: Rihards, naive implementation
#dta_clean = dta_clean.fillna(value=0, axis=1)
dta_clean = dta_clean.dropna()
dta_clean = dta_clean.reindex_axis(sorted(dta_clean.columns), axis=1)
#clean up data from zero rows 
for colname, colvalue in dta_clean.iteritems():
    if colname != 'facenumber_in_poster':
        dta_clean = dta_clean[dta_clean[colname] != 0]
dta_clean.count()
#clasify the data for the logistic regression
def label_gross (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 10000000)) : return 2
    elif ((gross >= 10000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 200000000)) : return 4
    elif (gross >= 200000000) : return 5

dta_clean['gross_class'] = dta_clean.gross.apply (lambda gross: label_gross (gross))


##svr classificaiton
y, X = dmatrices('gross_class ~  budget + C(language) + cast_total_facebook_likes + \
                 director_facebook_likes + duration + facenumber_in_poster + imdb_score + \
                 movie_facebook_likes + num_critic_for_reviews + num_user_for_reviews + num_voted_users + title_year',
                 dta_clean, return_type="dataframe")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = SVR(kernel='rbf', verbose=2)
model = model.fit(X, y)
# check the accuracy on the training set
model.score(X, y)

[LibSVM]

0.84855652575620177

In [ ]:
##svr regression
y, X = dmatrices('gross ~  budget + C(language) + cast_total_facebook_likes + \
                 director_facebook_likes + duration + facenumber_in_poster + imdb_score + \
                 movie_facebook_likes + num_critic_for_reviews + num_user_for_reviews + num_voted_users + title_year',
                 dta_clean, return_type="dataframe")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = SVR(kernel='linear', verbose=2)
model = model.fit(X, y)
# check the accuracy on the training set
model.score(X, y)

[LibSVM]